In [ ]:
import numpy as np
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Activation, Dropout, Flatten, Dense
from keras import applications
from keras.layers import Conv2D, MaxPooling2D

In [ ]:
img_width, img_height = 224, 224

top_model_weights_path = 'bottleneck_fc_model_multi_40epoch_6class_3lyr.h5'
train_data_dir = 'pix3d-ML/train'
validation_data_dir = 'pix3d-ML/validation'
test_data_dir = 'pix3d-ML/test'
botfet_train = 'bottleneck_features_train-4class.npy'
botfet_validation = 'bottleneck_features_validation-4class.npy'
num_classes = 4
t = [920,1097,1056,1743]#1049,1245,
z = [192,254,244,350] #sum must be div by 16 #249,244,
nb_train_samples = sum(t)
nb_validation_samples = sum(z)
epochs = 40
batch_size = 16

In [ ]:
def save_bottleneck_features():
    datagen = ImageDataGenerator(rescale=1. / 255,
                                 rotation_range=40,
                                 width_shift_range=0.2,
                                 height_shift_range=0.2,
                                 shear_range=0.2,zoom_range=0.2,fill_mode='nearest')
    datagen_v = ImageDataGenerator(rescale=1. / 255)
    # build the VGG16 network
    model = applications.VGG16(include_top=False, weights='imagenet')
    print('model loaded')
    generator = datagen.flow_from_directory(
        train_data_dir,
        target_size=(img_width, img_height),
        batch_size=batch_size,
        class_mode=None,
        shuffle=False)
    bottleneck_features_train = model.predict_generator(
        generator, nb_train_samples // batch_size)
#     with open('bottleneck_features_train.npy', 'wb') as f:
#         np.save(f,bottleneck_features_train)
    np.save(open(botfet_train, 'wb'),
            bottleneck_features_train)
    print(bottleneck_features_train.shape)
    generator = datagen_v.flow_from_directory(
        validation_data_dir,
        target_size=(img_width, img_height),
        batch_size=batch_size,
        class_mode=None,
        shuffle=False)
    bottleneck_features_validation = model.predict_generator(
        generator, nb_validation_samples // batch_size)
    np.save(open(botfet_validation, 'wb'),
            bottleneck_features_validation)
#     with open('bottleneck_features_validation.npy', 'wb') as f:
#         np.save(f,bottleneck_features_validation)
    print('done')

In [ ]:
save_bottleneck_features() #need to be done once

In [ ]:
def train_top_model(cn0,cn1,cn2,dr0,dr1,dr2,outp):
    train_data = np.load(open(botfet_train,'rb'))
#     print(train_data.shape)
    train_labels = []
    tf = []
    for en,item in enumerate(t):
        ii = [0]*num_classes
        ii[en] = 1
#         print(ii,item)
        for tt in range(item):
            train_labels.append(ii)
            tf.append(en)

    train_labels = np.array(train_labels)


    validation_data = np.load(open(botfet_validation,'rb'))
#     validation_labels = np.array([0]*za+[1]*zb+[2]*zc)
    validation_labels = []
    for en,item in enumerate(z):
        ii = [0]*num_classes
        ii[en] = 1
#         print(ii,item)
        for tt in range(item):
            validation_labels.append(ii)
            
    validation_labels = np.array(validation_labels)
#         [0] * int(nb_validation_samples / 2) + [1] * int(nb_validation_samples / 2))
    print(validation_data.shape,validation_labels.shape)
    model = Sequential()
    model.add(Flatten(input_shape=train_data.shape[1:]))
#     model.add(Dense(cn0, activation='relu'))
#     model.add(Dropout(dr0))
    model.add(Dense(cn1, activation='relu'))
    model.add(Dropout(dr1))
    model.add(Dense(cn2, activation='relu'))
    model.add(Dropout(dr2))
    model.add(Dense(num_classes, activation='softmax'))

    model.compile(optimizer='adam',
                  loss='categorical_crossentropy', metrics=['accuracy'])

    model.fit(train_data, train_labels,
              epochs=epochs,
              batch_size=batch_size,
              validation_data=(validation_data, validation_labels))
    model.save_weights(outp)
#     trill(model)
    return model
#     print('done')

In [ ]:
bbb = [64,128,256,512]
for dr0 in [0.2,0.3,0.4,0.5]:
    for dr1 in [0.2,0.3,0.4,0.5]:
        for dr2 in [0.2,0.3,0.4,0.5]:
            for it0 in range(len(bbb)):
                for it1 in range(it0,len(bbb)):
                    for it2 in range(it1,len(bbb)):
                        print('#####################################3')
                        train_top_model(bbb[it0],bbb[it1],bbb[it2],dr0,dr1,dr2)
                        print(bbb[it0],bbb[it1],bbb[it2],dr0,dr1,dr2)
    #                 print('#####################################3')

In [ ]:
#store output of previous cell in data-long.txt
file = open('data-long.txt','r')
ff = file.readlines()
# print(ff)

for z,f in enumerate(ff):
    if '#########' in f:
        # print(ff[z-1],ff[z-2])
        pp = ff[z-2].split(' ')
        pp1 = ff[z-1].strip().split(' ')
        # print(pp,pp1)
        pp1 = list(map(float, pp1))
        if float(pp[16])>.90 and float(pp[10])>.90: 
            print(ff[z-2],pp1) #prints parameters which has >.9 train and test acc
            # we can plot this data as well